<a href="https://colab.research.google.com/github/averma12/DL-from-scratch/blob/master/Training_loop_Learner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import tensor

In [0]:
a = torch.randn(5000,784)

In [4]:
a.unsqueeze(-1).shape

torch.Size([5000, 784, 1])

In [0]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle,gzip,math,torch,matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL = "http://deeplearning.net/data/mnist/mnist.pkl"

In [7]:
path = datasets.download_data(MNIST_URL, ext='.gz')
path

PosixPath('/root/.fastai/data/mnist.pkl.gz')

In [0]:
with gzip.open(path, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [9]:
x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))
n,c = x_train.shape
x_train, x_train.shape, y_train, y_train.shape, y_train.min(), y_train.max()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([50000, 784]),
 tensor([5, 0, 4,  ..., 8, 4, 8]),
 torch.Size([50000]),
 tensor(0),
 tensor(9))

In [0]:
def normalize(x, m, s): return (x-m)/s

In [12]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalize(x_train, train_mean, train_std)

In [0]:
x_valid = normalize(x_valid, train_mean, train_std)

In [15]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [0]:
from torch.nn import init

In [18]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [0]:
nh=50
b1 = torch.zeros(nh)
b2 = torch.zeros(1)

In [0]:
def lin(x, w, b): return x@w + b

In [0]:
def relu(x): return x.clamp_min(0.)

In [0]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))
w2 = torch.randn(nh,1)/math.sqrt(nh)

In [0]:
def relu(x): return x.clamp_min(0.) - 0.5

In [0]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [30]:
model(x_valid).shape

torch.Size([10000, 1])

In [0]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train,y_valid = y_train.float(),y_valid.float()

In [0]:
preds = model(x_train)

In [34]:
preds.shape

torch.Size([50000, 1])

In [35]:
mse(preds, y_train)

tensor(28.0870)

In [37]:
preds.squeeze().shape

torch.Size([50000])

In [48]:
((preds.squeeze() - y_train).unsqueeze(-1)/50000)

tensor([[-1.0677e-04],
        [ 1.4980e-05],
        [-9.3544e-05],
        ...,
        [-1.5273e-04],
        [-7.1954e-05],
        [-1.4841e-04]])

In [0]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x